In [14]:
import polars as pl

In [15]:
event_df = pl.read_csv("../data/events.csv")
category_tree_df = pl.read_csv("../data/category_tree.csv")
items_df = pl.read_csv("../data/item_properties.csv")

In [16]:
event_df.filter(pl.col('visitorid') == 1308708)

timestamp,visitorid,event,itemid,transactionid
i64,i64,str,i64,str
1434389834443,1308708,"""transaction""",181973,"""3529"""
1434391470175,1308708,"""view""",260955,null
1434389604062,1308708,"""view""",181973,null
1434389512630,1308708,"""view""",169256,null
1434389748591,1308708,"""addtocart""",181973,null


In [17]:
# Create list of customers who made a purchase
customers = event_df.filter(pl.col("transactionid").is_not_null()).select("visitorid").unique().to_series().to_list()
customers

[303466,
 1248805,
 1383834,
 1161252,
 1068179,
 937494,
 1044986,
 1289841,
 6699,
 248925,
 1213423,
 46535,
 239362,
 850983,
 153780,
 233979,
 697629,
 875891,
 663411,
 124269,
 240800,
 895690,
 883355,
 1125459,
 483303,
 172132,
 458788,
 383168,
 782163,
 54017,
 470316,
 1227440,
 48896,
 914152,
 552123,
 30824,
 446206,
 1265850,
 1083923,
 946268,
 800777,
 389069,
 1251955,
 1072496,
 105795,
 319457,
 196114,
 520400,
 661580,
 805761,
 658299,
 1088103,
 496796,
 1166352,
 1359706,
 864738,
 596678,
 806014,
 427854,
 865938,
 1327578,
 295588,
 510429,
 1390774,
 1368742,
 1220101,
 1031618,
 617793,
 39985,
 299259,
 1256293,
 1296388,
 119827,
 682534,
 1157313,
 884534,
 953625,
 104866,
 373852,
 413313,
 306738,
 1083658,
 314747,
 1067646,
 1203467,
 819001,
 195721,
 1085620,
 2242,
 1084569,
 1233201,
 1364157,
 957284,
 805630,
 834736,
 554737,
 373992,
 1132402,
 120601,
 1141334,
 671926,
 801313,
 1404130,
 69880,
 782961,
 1280004,
 279466,
 203974,
 11

In [18]:
# Creating a list of lists for every item a customer purchased 
purchases = []

for i in customers:
        items = event_df.filter((pl.col("visitorid") == pl.lit(i)) & (pl.col("transactionid").is_not_null())
                                ).select("itemid").to_series().to_list()    
        purchases.append(items)

In [19]:
purchases

[[257040],
 [273872, 234758, 303058, 259092, 279310],
 [200678],
 [104560, 44631],
 [366415],
 [405770],
 [441852],
 [11976],
 [51354, 280375, 210087, 73602],
 [456912, 135976],
 [276527, 455638, 185863, 280752, 255939, 185863],
 [286731, 119639, 303790, 380440],
 [227790],
 [459664],
 [181188, 448453],
 [398397,
  281854,
  433697,
  328298,
  442150,
  245810,
  364875,
  242372,
  385520,
  173223,
  81434,
  153778],
 [335054],
 [369447],
 [418609],
 [382787, 44888],
 [442423],
 [362697],
 [265934],
 [259112],
 [223203],
 [376671],
 [35202, 35202, 35202],
 [219300],
 [197455],
 [362760],
 [247698],
 [195423],
 [318333],
 [109427],
 [347549, 307931, 74144],
 [413951],
 [101238, 173806, 375207],
 [200514],
 [171555],
 [259884, 449391, 78268],
 [249434],
 [369933],
 [340921],
 [162139],
 [94773],
 [300594],
 [445115],
 [271393],
 [55120, 276324],
 [382945],
 [440892],
 [346088],
 [374996],
 [319905],
 [64137, 89044],
 [3854, 104929],
 [400093],
 [161354, 450276],
 [32267],
 [160441],


In [34]:
# Finding 5 items that a customer purchased along with a given item 
def recommendations(item_id, items_purchased):
    recommendations =[]
    for i in items_purchased:
        if item_id in i:
            recommendations +=i
    # Eliminate the item customer is currently viewed 
    # & Make sure all items are unique 
    recommendations = list(set(recommendations) - set([item_id]))
    return recommendations[:5]

In [35]:

recommendations(31640, purchases)

[424193, 246532, 217605, 149767, 124938]